In [1]:
import sys
sys.path.append('./flash3d')
sys.path.append('./flash3d/flash3d')
from flash3d.generator_test import Flash3DReconstructor
import torch
import types
from PIL import Image
from torchvision.utils import save_image
from torchvision.transforms.functional import to_pil_image
from flash3d.flash3d.util.export_param import postprocess
import numpy as np
from torchvision import transforms
from vlm_diffusion_pipeline import main as generate_diffusion_img
from matplotlib import pyplot as plt
from diffusers.utils import load_image, make_image_grid

/scratch/kf09/lz1278/lyenv/flash3d/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the path to the input image and the output directory, the input image is the very first image of the whole pipeline
intial_img_path = './flash3d/frame000652.jpg'
output_path = './flash3d-output'
current_directory = './flash3d-cache'

In [3]:
# Define the model, from Jiaqi's code at /flash3d/generator_test.py
flash3dreconstructor = Flash3DReconstructor()

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.2.2+cu121 with CUDA 1201 (you have 2.2.2)
    Python  3.11.8 (you have 3.11.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available


Instantiate: dinov2_vitl14


In [4]:
# Define the decorator to add the function to the instance of the reconstructor
def decorator_add_function_to_instance(instance, func):
    setattr(instance, func.__name__, types.MethodType(func, instance))

setattr(flash3dreconstructor, 'add_function_to_instance', types.MethodType(decorator_add_function_to_instance, flash3dreconstructor))

# Preprocess the input image, get the flash3d output, then the postprocess
@flash3dreconstructor.add_function_to_instance
def flash3d_postprocess(self, index, image):
    if (index == 0):
        image = self.to_tensor(image).to(self.device).unsqueeze(0)

    else:
        image = self.diffusion_img

    save_image(image, current_directory+f"/imgs/{index}_inputpre.png")
    self.gt_img.append(image)

    inputs = {
        ("color_aug", 0, 0): image,
    }

    # flash3d输出
    result = self.model(inputs)

    # 2层3dg
    outputs = postprocess(result,
                        num_gauss=2,
                        h=self.cfg.dataset.height,
                        w=self.cfg.dataset.width,
                        pad=self.cfg.dataset.pad_border_aug)
    # 1层3dg
    outputs_1_gauss = postprocess(result,
                        num_gauss=1,
                        h=self.cfg.dataset.height,
                        w=self.cfg.dataset.width,
                        pad=self.cfg.dataset.pad_border_aug)
    
    return outputs, outputs_1_gauss

# 处理初始输入图片，添加到地图中
# Directly copy from Jiaqi's code
@flash3dreconstructor.add_function_to_instance
def flash3d_initial_map(self, outputs, outputs_1_gauss):
    self.map_param_2 = outputs
    self.map_param_2['rotations'] = torch.tensor(self.map_param_2['rotations']).to('cuda')

    self.map_param_1 = outputs_1_gauss
    self.map_param_1['rotations'] = torch.tensor(self.map_param_1['rotations']).to('cuda')

    self.map_param_2 = self.optimize_map(self.map_param_2)

# 处理生成的图片，按照mask添加新元素，变换到世界坐标下，添加到地图中
# Directly copy from Jiaqi's code
@flash3dreconstructor.add_function_to_instance
def flash3d_additional_map(self, index, outputs, outputs_1_gauss):
    w2c = self.w2c[index]

    self.cur_param_2 = outputs
    self.cur_param_2['rotations'] = torch.tensor(self.cur_param_2['rotations']).to('cuda')

    self.cur_param_1 = outputs_1_gauss
    self.cur_param_1['rotations'] = torch.tensor(self.cur_param_1['rotations']).to('cuda')

    c2w = torch.inverse(w2c)
    self.cur_param_1['means'] = self.apply_transformation(self.cur_param_1['means'], c2w.to('cuda'))
    self.cur_param_2['means'] = self.apply_transformation(self.cur_param_2['means'], c2w.to('cuda'))

    # 保留新增的部分
    mask = ~torch.tensor(self.mask).view(-1)
    mask_2 = mask.repeat(2)
    # update global map
    for key in self.map_param_1.keys():

        original_tensor = self.map_param_1[key].to('cuda')
        updated_tensor = torch.tensor(self.cur_param_1[key]).to('cuda')
        updated_tensor = updated_tensor[mask]

        if isinstance(updated_tensor, np.ndarray):
            updated_tensor = torch.tensor(updated_tensor).to('cuda')

        self.map_param_1[key] = torch.cat((original_tensor, updated_tensor), dim=0)

        original_tensor = self.map_param_2[key].to('cuda')
        updated_tensor = torch.tensor(self.cur_param_2[key]).to('cuda')
        updated_tensor = updated_tensor[mask_2]

        if isinstance(updated_tensor, np.ndarray):
            updated_tensor = torch.tensor(updated_tensor).to('cuda')

        self.map_param_2[key] = torch.cat((original_tensor, updated_tensor), dim=0)

    self.map_param_2 = self.optimize_map(self.map_param_2)

@flash3dreconstructor.add_function_to_instance
def flash3d_prepare_img_mask_for_diffusion(self, index):
    # 新视角下渲染
    w2c = self.w2c[index+1]
    im_original, radius = self.renderer.render(self.map_param_2, w2c)
    im = im_original[:, 32:352, 32:608]
    self.renderer.save_image(im, current_directory+f"/imgs/{index}_render_2gauss.png")

    # render 1 gauss per pixel
    im_1_gauss_original, radius = self.renderer.render(self.map_param_1, w2c)
    im_1_gauss = im_1_gauss_original[:, 32:352, 32:608]
    self.renderer.save_image(im_1_gauss, current_directory+f"/imgs/{index}_render_1gauss.png")

    image_a_pil = to_pil_image(im_1_gauss)
    image_b_pil = to_pil_image(im)
    masked_img, mask = self.apply_mask_from_images(image_a_pil, image_b_pil)

    self.mask = mask # mask for the diffusion and adding new 3dg
    mask_render_path = current_directory+f"/imgs/{index}_masked_rendered.png"
    self.renderer.save_image(masked_img, mask_render_path)

    # 获取diffusion的mask
    image_a_pil = to_pil_image(im_1_gauss_original)
    image_b_pil = to_pil_image(im_original)
    masked_img_diffusion, mask_diffusion = self.apply_mask_from_images(image_a_pil, image_b_pil)
    mask_diffusion = ~torch.tensor(mask_diffusion)
    mask_diffusion = mask_diffusion.to(torch.float32)


    # input of diffusion
    mask_render_path_diffusion = current_directory+f"/imgs/{index}_masked_rendered_original.png"
    self.renderer.save_image(masked_img_diffusion, mask_render_path_diffusion)
    mask_path_diffusion = current_directory+f"/imgs/{index}_mask_diffusion.png"
    self.renderer.save_image(mask_diffusion, mask_path_diffusion)

    return mask_render_path_diffusion, mask_path_diffusion

@flash3dreconstructor.add_function_to_instance
def flash3d_post_process_diffusion_img(self, diffusion_img):
    transform = transforms.Compose([
                transforms.Resize((384, 640)),  # 先调整大小为 (height, width)
                transforms.CenterCrop((320, 576)),  # 再中心裁剪为 (height, width)
                transforms.Pad(padding=32, fill=(0, 0, 0))  # 最后添加 32 像素的填充，填充颜色为黑色
            ])
    diffusion_img = transform(diffusion_img)

    self.diffusion_img = self.to_tensor(diffusion_img).to(self.device).unsqueeze(0) # [1, 3, 384, 640]

@flash3dreconstructor.add_function_to_instance
def flash3d_final_process(self, ):
    reconstructor = self
    # 优化1 layer的map
    reconstructor.map_param_1 = reconstructor.optimize_map(reconstructor.map_param_1)

    # 不同视角渲染地图并保存
    for i in range(15, -1, -1):
        temp_w2c = reconstructor.get_SE3_rotation_y(i)

        im, radius = reconstructor.renderer.render(reconstructor.map_param_1, temp_w2c)
        im = im[:, 32:352, 32:608]
        reconstructor.renderer.save_image(im, current_directory+f'/rotate_demo/{15-i}_render.png')

    for i in range(0, 30, 1):
        temp_w2c = reconstructor.get_SE3_rotation_y(i)

        im, radius = reconstructor.renderer.render(reconstructor.map_param_1, temp_w2c)
        im = im[:, 32:352, 32:608]
        reconstructor.renderer.save_image(im, current_directory+f'/rotate_demo/{i+16}_render.png')

In [5]:
# 初始视角(input image的视角)
w2c_0 = torch.tensor([
                    [1.0, 0.0, 0.0, 0.0],  
                    [0.0, 1.0, 0.0, 0.0], 
                    [0.0, 0.0, 1.0, 0.0], 
                    [0.0, 0.0, 0.0, 1.0]
                ], dtype=torch.float32)

# w2c back denotes the transformation matrix for the camera to backward while maintaining the same view angle
# 0.2 is roungly the distance (not entirely sure) between the camera and the object, adjust this value to adjust the distance between the camera and the object
# If want to combine it with rotation, just multiply the rotation matrix with this matrix
backward_distance = 0.2
w2c_back = torch.tensor([
                    [1.0, 0.0, 0.0, 0.0],  
                    [0.0, 1.0, 0.0, 0.0], 
                    [0.0, 0.0, 1.0, backward_distance], 
                    [0.0, 0.0, 0.0, 1.0]
                ], dtype=torch.float32)

# 添加视角，w2c_0为初始视角
flash3dreconstructor.w2c.append(w2c_0)

In [6]:
# Add the transformation matrix you want to apply to the camera
rotate_angle = [10, 20, 30]
for angle in rotate_angle:
    rotate_matrix = flash3dreconstructor.get_SE3_rotation_y(angle)
    flash3dreconstructor.w2c.append(rotate_matrix) # This line to add the rotation matrix to the camera
# rotate_matrix = flash3dreconstructor.get_SE3_rotation_y(rotate_angle)
# flash3dreconstructor.w2c.append(rotate_matrix) # This line to add the rotation matrix to the camera

RuntimeError: No CUDA GPUs are available

In [7]:
# Prepare the input image, which refers to the very first image of the whole pipeline
img = Image.open(intial_img_path).convert("RGB")
flash3dreconstructor.check_input_image(img)
img = flash3dreconstructor.preprocess(img, dynamic_size=True, padding=True)

In [35]:
# Define the current loop index, the loop includes the entire pipeline, from the input image to the final output, it normally depends on the number of transformations
# you add to the w2c list of the reconstructor
# Note the input image of the first loop (current_loop_index = 0) is the image you prepared above
# The output image of the first loop is the input image of the second loop, and so on
current_loop_index = 3

In [36]:
flash3dreconstructor.model.to('cuda')

outputs, outputs_1_gauss = flash3dreconstructor.flash3d_postprocess(current_loop_index, img)
if current_loop_index == 0:
    flash3dreconstructor.flash3d_initial_map(outputs, outputs_1_gauss)
else:
    flash3dreconstructor.flash3d_additional_map(current_loop_index, outputs, outputs_1_gauss)

if current_loop_index + 1 < len(flash3dreconstructor.w2c):
    mask_render_path_diffusion, mask_path_diffusion = flash3dreconstructor.flash3d_prepare_img_mask_for_diffusion(current_loop_index)

flash3dreconstructor.model.to('cpu')

C:\Users\UTastePC\AppData\Local\Temp\ipykernel_11072\2593881491.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  updated_tensor = torch.tensor(self.cur_param_1[key]).to('cuda')
C:\Users\UTastePC\AppData\Local\Temp\ipykernel_11072\2593881491.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  updated_tensor = torch.tensor(self.cur_param_2[key]).to('cuda')
d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\./flash3d\renderer.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w2c = torch.tensor(w2c).cuda().float()


Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_0_start_render.png
Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_1_start_render.png
Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_2_start_render.png
Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_3_start_render.png
Tracking Iteration 1, Loss: 4552.4761
Tracking Iteration 2, Loss: 4954.2080
Tracking Iteration 3, Loss: 4494.5562
Tracking Iteration 4, Loss: 4348.6875
Tracking Iteration 5, Loss: 4247.7188
Tracking Iteration 6, Loss: 4126.9707
Tracking Iteration 7, Loss: 4036.7437
Tracking Iteration 8, Loss: 3974.5220
Tracking Iteration 9, Loss: 3909.1885
Tracking Iteration 10, Loss: 3849.4834
Tracking Iteration 11, Loss: 3795.8528
Tracking Iteration 12, Loss: 3749.8350
Tracking Iteration 13, Loss: 3707.6917
Tracking Iteration 14, Loss: 3667.7437
Tracking Iteration 15, Loss: 3634.6392


GaussianPredictor(
  (models): ModuleDict(
    (unidepth_extended): UniDepthExtended(
      (unidepth): UniDepthDepth(
        (depth_prediction_model): UniDepthV1(
          (pixel_encoder): DinoVisionTransformer(
            (patch_embed): PatchEmbed(
              (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
              (norm): Identity()
            )
            (blocks): ModuleList(
              (0-23): 24 x NestedTensorBlock(
                (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
                (attn): MemEffAttention(
                  (qkv): Linear(in_features=1024, out_features=3072, bias=True)
                  (attn_drop): Dropout(p=0.0, inplace=False)
                  (proj): Linear(in_features=1024, out_features=1024, bias=True)
                  (proj_drop): Dropout(p=0.0, inplace=False)
                )
                (ls1): LayerScale()
                (drop_path1): Identity()
                (norm2): LayerNorm((1024,),

In [37]:
# Display the render image and mask image
if current_loop_index + 1 < len(flash3dreconstructor.w2c):
    rendered_img = Image.open(mask_render_path_diffusion)
    mask_img = Image.open(mask_path_diffusion)

    grid_img = make_image_grid([rendered_img, mask_img], rows=1, cols=2)
    plt.imshow(grid_img)
    plt.axis('off')
    plt.savefig(f'./imgs/before_inpainting_{current_loop_index}.jpg')

In [38]:
Prompt_diffusion = 'A indoor scene, a room, a window, two sofas.'

In [39]:
if current_loop_index + 1 < len(flash3dreconstructor.w2c):
    diffusion_img = generate_diffusion_img(image_path=mask_render_path_diffusion, mask_path=mask_path_diffusion,
                                                    prompt_question=None,
                                                    prompt_diffusion=Prompt_diffusion,
                                                    base_model='stable-diffusion-v2', index=current_loop_index, strength=1.0,
                                                    negative_prompt="bad architecture, inconsistent, poor details, blurry") # 512*512

In [40]:
# Display the diffusion image
if current_loop_index + 1 < len(flash3dreconstructor.w2c):
    plt.imshow(diffusion_img)
    plt.axis('off')
    plt.savefig(f'./imgs/diffusion_{current_loop_index}.jpg')

In [41]:
flash3dreconstructor.model.to('cuda')

GaussianPredictor(
  (models): ModuleDict(
    (unidepth_extended): UniDepthExtended(
      (unidepth): UniDepthDepth(
        (depth_prediction_model): UniDepthV1(
          (pixel_encoder): DinoVisionTransformer(
            (patch_embed): PatchEmbed(
              (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
              (norm): Identity()
            )
            (blocks): ModuleList(
              (0-23): 24 x NestedTensorBlock(
                (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
                (attn): MemEffAttention(
                  (qkv): Linear(in_features=1024, out_features=3072, bias=True)
                  (attn_drop): Dropout(p=0.0, inplace=False)
                  (proj): Linear(in_features=1024, out_features=1024, bias=True)
                  (proj_drop): Dropout(p=0.0, inplace=False)
                )
                (ls1): LayerScale()
                (drop_path1): Identity()
                (norm2): LayerNorm((1024,),

In [42]:
# Postprocess the diffusion image
flash3dreconstructor.flash3d_post_process_diffusion_img(diffusion_img)

In [43]:
raise ValueError('Stop Here')

ValueError: Stop Here

In [44]:
flash3dreconstructor.flash3d_final_process()

Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_0_start_render.png
Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_1_start_render.png
Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_2_start_render.png
Image saved as d:\Local\Courses\COMP8536\3dgs-utils\ANU-COMP8536-2024s2\flash3d/imgs/0_3_start_render.png
Tracking Iteration 1, Loss: 111754.4844
Tracking Iteration 2, Loss: 103192.8438
Tracking Iteration 3, Loss: 92774.3828
Tracking Iteration 4, Loss: 81474.3750
Tracking Iteration 5, Loss: 70132.5000
Tracking Iteration 6, Loss: 59640.6094
Tracking Iteration 7, Loss: 50654.8711
Tracking Iteration 8, Loss: 43435.9297
Tracking Iteration 9, Loss: 37905.4297
Tracking Iteration 10, Loss: 33802.8203
Tracking Iteration 11, Loss: 30790.3516
Tracking Iteration 12, Loss: 28571.1602
Tracking Iteration 13, Loss: 26914.5469
Tracking Iteration 14, Loss: 25650.1621
Tracking Iteration 15, 